In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [3]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

headings = ['Postcode', 'Borough', 'Neighbourhood']
for table in tables:
    current_headings = table.columns.values[:3]
    if list(current_headings) == headings:
        break
table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
table.shape

(287, 3)

In [5]:
index=0
while index<table.shape[0]:
    
    if(table.iloc[index,1]=='Not assigned'):
        table.drop(index,inplace=True)
        table.reset_index(inplace=True,drop=True)
        index=index-1
    index=index+1
        
     
        

In [6]:
table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [7]:
i=0;j=0
while i<table.shape[0]:
    j=0
    while j<table.shape[0]:
        if(table.iloc[i,0]==table.iloc[j,0] and i!=j):
            table.iloc[i,2]=table.iloc[i,2]+','+table.iloc[j,2]
            table.drop(j,inplace=True)
            table.reset_index(inplace=True,drop=True)
            j=j-1
        j=j+1
    i=i+1

In [8]:
table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [9]:
for i in range(table.shape[0]):
    if(table.iloc[i,2]=='Not assigned'):
        table.iloc[i,2]=table.iloc[i,1]

In [10]:
table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
table.shape

(103, 3)

In [12]:
table['Latitude']=None

In [13]:
table['Longitude']=None

In [14]:
table.columns

Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [15]:
table.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,Harbourfront,None,None
3,M6A,North York,"Lawrence Heights,Lawrence Manor",None,None
4,M7A,Queen's Park,Queen's Park,None,None


In [16]:
co=pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

In [17]:
co.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
i=0;j=0
while i<table.shape[0]:
    j=0
    while j<co.shape[0]:
        if(table.iloc[i,0]==co.iloc[j,0]):
            table.iloc[i,3]=co.iloc[j,1]
            table.iloc[i,4]=co.iloc[j,2]
        j=j+1
    i=i+1
        

In [19]:
table.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
